In [1]:
from utils import *
from metrics import *
from preprocess_data import *
# from row_resnet2d import resnet18 as myresnet18
from proposed_resnet2d import resnet18 as myresnet18
from resnet2d import resnet18 as resnet18

/home/rachwork0/miniconda3/envs/project/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/rachwork0/miniconda3/envs/project/lib/python3.8/site-packages/tltorch/factorized_tensors/core.py:145: UserWarning: Creating a subclass of FactorizedTensor TensorizedTensor with no name.
  warnings.warn(f'Creating a subclass of FactorizedTensor {cls.__name__} with no name.')


In [2]:
import torch
import matplotlib.pyplot as plt

In [3]:
DEVICE = 'cuda:0'

mymodel = torch.load('./result/proposed_model.pth').to(DEVICE)
model = torch.load('./result/model.pth').to(DEVICE)

In [6]:
### Sets and clip is indicate a single video clip
frames, frame_labels = get_data('./dataset', sets='02', clip='069')

print(f"Frame: {len(frames)} / Labels: {len(frame_labels)}")

Frame: 53 / Labels: 53


In [7]:
my_result = []
result = []
for frame, labels in zip(frames, frame_labels):
    inputs = frame.unsqueeze(0).to(DEVICE)
    my_prediction = mymodel(inputs)
    my_prediction = my_prediction.cpu().detach()[0][0].numpy().tolist()
    
    prediction = model(inputs)
    prediction = prediction.cpu().detach()[0].numpy().tolist()
    my_result.append([my_prediction[0], my_prediction[1], labels[0], labels[1]])
    result.append([prediction[0], prediction[1], labels[0], labels[1]])

In [8]:
my_results = np.array(my_result)
results = np.array(result)
# results

In [9]:
# plt.figure(figsize=(12, 4))
# plt.subplot(1, 2, 1)
# plt.plot(my_results[:, 0], label='Prediction')
# plt.plot(my_results[:, 2], label='Truth')
# plt.title('Arousal')
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.plot(my_results[:, 1], label='Prediction')
# plt.plot(my_results[:, 3], label='Truth')
# plt.title('Valence')
# plt.legend()

# plt.figure(figsize=(12, 4))
# plt.subplot(1, 2, 1)
# plt.plot(results[:, 0], label='Prediction')
# plt.plot(results[:, 2], label='Truth')
# plt.title('Arousal')
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.plot(results[:, 1], label='Prediction')
# plt.plot(results[:, 3], label='Truth')
# plt.title('Valence')
# plt.legend()

# Test results of models

In [10]:
batch_size = 256
train_idx, valid_idx, test_idx = get_label('./dataset', 11)

training_dataloader = DataLoader(AFEWDataset(train_idx), batch_size = batch_size, shuffle=True)
validation_dataloader = DataLoader(AFEWDataset(valid_idx), batch_size = batch_size*2)
test_dataloader = DataLoader(AFEWDataset(test_idx), batch_size = batch_size*2)

In [11]:
_, val_arousal, val_valence = evaluation(validation_dataloader, mymodel, DEVICE)  
_, test_arousal, test_valence = evaluation(test_dataloader, mymodel, DEVICE)   

print("------------------------------------------------------------")
print("2D HO-CPConv (Proposed)")
print("------------------------------------------------------------")
print(f"Validation|| Arousal:{val_arousal:.4f} Valence:{val_valence:.4f}")
print(f"Test|| Arousal:{test_arousal:.4f} Valence:{test_valence:.4f}")


------------------------------------------------------------
2D HO-CPConv (Proposed)
------------------------------------------------------------
Validation|| Arousal:0.2578 Valence:0.2373
Test|| Arousal:0.2110 Valence:0.2518


In [ ]:
_, val_arousal, val_valence = evaluation(validation_dataloader, model, DEVICE)  
_, test_arousal, test_valence = evaluation(test_dataloader, model, DEVICE)   

print("------------------------------------------------------------")
print("2D ResNet")
print("------------------------------------------------------------")
print(f"Validation|| Arousal:{val_arousal:.4f} Valence:{val_valence:.4f}")
print(f"Test|| Arousal:{test_arousal:.4f} Valence:{test_valence:.4f}")


# Functions  

In [4]:
def evaluation(dataloader, model, DEVICE):
    ''' Evaluate the model with validation and test data (for my implementation'''
    result = []
    with torch.no_grad():
        model.eval()
        for i, data in enumerate(dataloader):
            inputs, labels = data
            outputs = model(inputs.to(DEVICE)).reshape(-1, 2)
            result.append(torch.hstack([outputs, labels.to(DEVICE)]))
       
        result = torch.vstack(result)
        arousal_rmse = RMSE(result[2], result[0])
        valence_rmse = RMSE(result[3], result[1])
    
    return result, arousal_rmse.item(), valence_rmse.item()


In [5]:
def get_data(root_dir, sets='01', clip='001'):

    
    single_data = []
    single_label = []

    videos = [i for i in os.listdir(f'./{root_dir}/{sets}') 
    if (not i.startswith('.')) and (i!= 'README.md')]
    videos.sort()
        # Videos
    path_to_dataset = f'./dataset/{sets}/{clip}/{clip}.json'
    f = open(path_to_dataset)
    data = json.load(f)
    frame_key = list(data['frames'].keys())
            # Frames

    tmp_labels = []
    for k in frame_key:
        img_dir = f'{sets}/{clip}/{k}.png'
        arousal = data['frames'][k]['arousal']
        valence = data['frames'][k]['valence']
        ld_coords = crop_landmark(data['frames'][k]['landmarks'])
        if ld_coords is None:
            pass
        else:
            x1, x2, y1, y2 = ld_coords
            img_name = os.path.join(root_dir, img_dir)
            img = Image.open(img_name)
            train_img = transform_test(img)
            cropped_img = train_img[:, x1:x2, y1:y2]
            single_data.append(cropped_img)
            single_label.append([arousal, valence])
        
    return single_data, single_label
